In [3]:
import "dotenv/config";

[Module: null prototype] { default: {} }

In [4]:
import { ChatOpenAI, OpenAIEmbeddings } from "@langchain/openai";

const model = new ChatOpenAI({
  modelName: "gpt-3.5-turbo-1106"
});

In [5]:
import { CheerioWebBaseLoader } from "langchain/document_loaders/web/cheerio";
import * as cheerio from "cheerio"

const url = "https://docs.smith.langchain.com/overview";

const loader = new CheerioWebBaseLoader(      
  url
);

const docs = await loader.load();


In [6]:
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";

const splitter = new RecursiveCharacterTextSplitter();
const splitDocs = await splitter.splitDocuments(docs);

In [7]:
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const embeddings = new OpenAIEmbeddings();
const vectorstore = await MemoryVectorStore.fromDocuments(
  splitDocs,
  embeddings
);

In [8]:
import { createStuffDocumentsChain } from "langchain/chains/combine_documents";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { StringOutputParser } from "@langchain/core/output_parsers";

const prompt = ChatPromptTemplate.fromTemplate(`Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}`);

const documentChain = await createStuffDocumentsChain({
  llm: model,
  prompt: prompt,
  outputParser: new StringOutputParser()
});